In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
%run glassdoor_scrape_tools.ipynb

In [16]:
# workday review - company culture 
url_culture = 'https://www.glassdoor.com/Reviews/Workday-Reviews-E197851.htm'
url_benefit = 'https://www.glassdoor.com/Benefits/Workday-US-Benefits-EI_IE197851.0,7_IL.8,10_IN1.htm'
url_interview = 'https://www.glassdoor.com/Interview/Workday-Interview-Questions-E197851.htm'

In [65]:
driver = webdriver.Chrome(executable_path=r"D:\\Dev\WebDriver\\chromedriver.exe")

C:\Users\User\AppData\Local\Temp\ipykernel_14616\245330835.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"D:\\Dev\WebDriver\\chromedriver.exe")


In [66]:
driver.get(url_benefit)

In [ ]:
# get benefit reviews
key_metrics = ['date', 'text', 'rating', 'position']
df_benefit = pd.DataFrame(columns = key_metrics)
df_benefit = get_benefit_reviews(key_metrics, df_benefit)

In [68]:
df_benefit

,date,text,rating,position
0,"Jan 31, 2023",Comprehensive and overall very generous benefi...,5.0,Current Employee
1,"Jan 31, 2023",Comprehensive package that includes RSUs and l...,5.0,"Former Account Executive in Dallas, TX, Texas"
2,"Jan 30, 2023","Great perks include unlimited PTO, office kitc...",5.0,Current Software Development Engineer in Pleas...
3,"Jan 26, 2023",they have really good benefits which are flexi...,5.0,Current Senior Software Engineer in Pleasanton...
4,"Jan 24, 2023",Great benefits for working parents,5.0,Current Senior Account Executive in San Franci...
...,...,...,...,...
463,"Oct 12, 2014","They do not match 401k yet, which is disappoin...",3.0,Current Associate Software Engineer in Pleasan...
464,"Sep 16, 2014",In today's business environment of cutting cos...,5.0,Current Employee
465,"Aug 7, 2014","RSU, ESPP, Gym membership reimbursement, and c...",5.0,"Current Linux administrator in Pleasanton, CA,..."
466,"Aug 14, 2014",The flexible scheduling and opportunity to wor...,4.0,Current Employee


In [69]:
df_benefit.to_excel('workday_raw_benefit_review.xlsx')

# Data Cleaning

In [74]:
df_benefit = pd.read_excel('workday_raw_benefit_review.xlsx')
df_benefit.drop(['Unnamed: 0'], axis = 1, inplace=True)
df_benefit.head()

,date,text,rating,position
0,"Jan 31, 2023",Comprehensive and overall very generous benefi...,5.0,Current Employee
1,"Jan 31, 2023",Comprehensive package that includes RSUs and l...,5.0,"Former Account Executive in Dallas, TX, Texas"
2,"Jan 30, 2023","Great perks include unlimited PTO, office kitc...",5.0,Current Software Development Engineer in Pleas...
3,"Jan 26, 2023",they have really good benefits which are flexi...,5.0,Current Senior Software Engineer in Pleasanton...
4,"Jan 24, 2023",Great benefits for working parents,5.0,Current Senior Account Executive in San Franci...


In [77]:
# re-organize 'date' column
from datetime import datetime
df_benefit['year'] = df_benefit.date.map(lambda x: x.split(',')[-1])
df_benefit.head()

,date,text,rating,position,year
0,"Jan 31, 2023",Comprehensive and overall very generous benefi...,5.0,Current Employee,2023
1,"Jan 31, 2023",Comprehensive package that includes RSUs and l...,5.0,"Former Account Executive in Dallas, TX, Texas",2023
2,"Jan 30, 2023","Great perks include unlimited PTO, office kitc...",5.0,Current Software Development Engineer in Pleas...,2023
3,"Jan 26, 2023",they have really good benefits which are flexi...,5.0,Current Senior Software Engineer in Pleasanton...,2023
4,"Jan 24, 2023",Great benefits for working parents,5.0,Current Senior Account Executive in San Franci...,2023


In [86]:
df_benefit['new date'] = df_benefit.date.map(lambda x: x.replace(',', ''))
df_benefit['new date'] = df_benefit['new date'].map(lambda x: datetime.strptime(x, '%b %d %Y').date())

In [88]:
df_benefit.drop(['date'], axis=1, inplace=True)
df_benefit.head()

,text,rating,position,year,new date
0,Comprehensive and overall very generous benefi...,5.0,Current Employee,2023,2023-01-31
1,Comprehensive package that includes RSUs and l...,5.0,"Former Account Executive in Dallas, TX, Texas",2023,2023-01-31
2,"Great perks include unlimited PTO, office kitc...",5.0,Current Software Development Engineer in Pleas...,2023,2023-01-30
3,they have really good benefits which are flexi...,5.0,Current Senior Software Engineer in Pleasanton...,2023,2023-01-26
4,Great benefits for working parents,5.0,Current Senior Account Executive in San Franci...,2023,2023-01-24


In [100]:
df_benefit['location'] = df_benefit.position.map(lambda x: x.split(',')[-1])
df_benefit['location'] = df_benefit.location.apply(lambda x: None if x == 'Current Employee' else x)

In [106]:
# create 'city' column
df_benefit['city'] = df_benefit.position.map(lambda x: x.split(',')[0].split(' ')[-1])
df_benefit['city'] = df_benefit.city.apply(lambda x: None if x == 'Employee' else x)
df_benefit.city = df_benefit.position.apply(lambda x: x.partition(' in ')[2] if 'in' in x else x)
df_benefit.city = df_benefit.city.map(lambda x : x.split(',')[0])

In [ ]:
# mark the status of employees: former/current
df_benefit['status'] = df_benefit.position.apply(status)

In [131]:
# clean the 'position' column
df_benefit.position = df_benefit.position.apply(position)
df_benefit.position = df_benefit.position.apply(lambda x: x.partition(' in ')[0] if 'in' in x else x)
df_benefit.position = df_benefit.position.apply(lambda x: ' '.join(x.split(' ')[1:]) if 'Current' in x or 'Former' in x else x)

In [138]:
df_benefit.rename(columns={"new date": "date", "location": "state"}, inplace=True)

In [146]:
df_benefit.head()

,text,rating,position,year,date,state,city,status
0,Comprehensive and overall very generous benefi...,5.0,unspecified,2023,2023-01-31,None,unspecified,Current
1,Comprehensive package that includes RSUs and l...,5.0,Account Executive,2023,2023-01-31,Texas,Dallas,Former
2,"Great perks include unlimited PTO, office kitc...",5.0,Software Development Engineer,2023,2023-01-30,California,Pleasanton,Current
3,they have really good benefits which are flexi...,5.0,Senior Software Engineer,2023,2023-01-26,California,Pleasanton,Current
4,Great benefits for working parents,5.0,Senior Account Executive,2023,2023-01-24,California,San Francisco,Current


In [147]:
df_benefit.to_excel('workday_raw_benefit_review.xlsx')